Extraindo e limapndo dados

In [ ]:
import pandas as pd 
import matplotlib.pyplot as plt

from sklearn.model_selection import (train_test_split,
                                     cross_validate,
                                      StratifiedKFold,)
from sklearn.metrics import (
    ConfusionMatrixDisplay,
    classification_report
)

from sklearn.feature_selection import mutual_info_classif
from imblearn.under_sampling import NearMiss
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline as imbpipeline




url = '/home/fagner/ProjetoAlura3/data/TelecomX_Data_Normalizado_limpo.csv'
df = pd.read_csv(url)

#print(df.info())
#print(df['remainder__churn'].value_counts())
#rint(df.corr)
df_clean = df.dropna()
print(df.shape)
print(df_clean.shape)


y = df_clean['remainder__churn']
x = df_clean.drop(axis=1, columns= 'remainder__churn')

mic = mutual_info_classif(x,y,discrete_features='auto') 

mic_df = pd.DataFrame({'Features': x.columns, 'MI': mic}).reset_index()
mic_df.sort_values('MI', ascending=False, inplace=True)
#print(mic_df)


limite = 0.01 
features_Relevantes = mic_df[mic_df['MI'] >= limite ]['Features'].tolist()
x_relevante = x[features_Relevantes]



#organizando dados 

''' 
dados relevantes em relação a x e y : 
 index                           Feature        MI
5       5         onehotencoder__contract_2  0.085471
24     24                 remainder__tenure  0.073468
6       6         onehotencoder__contract_3  0.055854
8       8    onehotencoder__paymentmethod_2  0.047189
3       3  onehotencoder__internetservice_2  0.040675
19     19          remainder__charges_total  0.039975
18     18        remainder__charges_monthly  0.035922
1       1  onehotencoder__internetservice_0  0.021691
17     17       remainder__paperlessbilling  0.019522
4       4         onehotencoder__contract_1  0.017663
14     14            remainder__techsupport  0.015361
11     11         remainder__onlinesecurity  0.014797
10     10    onehotencoder__paymentmethod_4  0.011757

paymentMethod{'Mailed check': 1, 'Electronic check': 2, 'Credit card (automatic)': 3, 'Bank transfer (automatic)': 4}
InternetService{'DSL': 1, 'Fiber optic': 2, 'no': 0}
Contract{one-year : 1, month-to-month:2 ,two-year : 3}

'''
# Removendo imports não utilizados
# Removidos: confusion_matrix, accuracy_score, precision_score, recall_score, f1_score


ImportError: cannot import name 'validate_data' from 'sklearn.utils.validation' (/home/fagner/ProjetoAlura3/venv/lib/python3.12/site-packages/sklearn/utils/validation.py)

Dividindo e validando

In [ ]:
def resultado(resultados):
    media = resultados['test_score'].mean()
    desvio_padrao = resultados['test_score'].std()
    print(f'Intervalo de confiança: [{media - 2*desvio_padrao}, {min(media + 2*desvio_padrao, 1)}]')


skf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 5)
x_t, x_teste, y_t, y_teste = train_test_split(x, y, test_size = 0.15, stratify = y, random_state = 5)


undersample = NearMiss(version=3)
x_balance, y_balance  = undersample.fit_resample(x_t,y_t)
modelo = DecisionTreeClassifier(max_depth = 10)
pipeline = imbpipeline(              [  ( 'undersampling',NearMiss(version=3)  ) , ('arvore',modelo )     ]       )
skf = StratifiedKFold(n_splits=5, shuffle = True, random_state = 5)
cv_resultados = cross_validate(pipeline, x_t, y_t, cv=skf, scoring = 'recall')
resultado(cv_resultados)


undersample = NearMiss(version=3)
x_balance, y_balance  = undersample.fit_resample(x_t,y_t)
modelo = DecisionTreeClassifier(max_depth=10)
modelo.fit(x_balance,y_balance)
y_prev = modelo.predict(x_teste)

print(classification_report(y_teste,y_prev))
ConfusionMatrixDisplay.from_predictions(y_teste,y_prev).plot()
plt.show()

Relatorio final

In [ ]:
# Análise do perfil do cliente que mais sai da empresa

# Informações extraídas dos dados relevantes (comentário do usuário):
# - Contrato: month-to-month (2) e two-year (3) têm alta relevância
# - Método de pagamento: Electronic check (2) é relevante
# - InternetService: Fiber optic (2) é relevante
# - Cobrança mensal e total são relevantes
# - Paperless billing (fatura digital) é relevante
# - Suporte técnico e segurança online têm influência

relatorio = """
Perfil do cliente com maior propensão a sair da empresa:

- Tipo de contrato: Clientes com contrato mensal (month-to-month) apresentam maior tendência de churn, pois têm maior flexibilidade para cancelar.
- Método de pagamento: Usuários que pagam via 'Electronic check' estão mais propensos a sair, indicando possível insatisfação ou menor engajamento digital.
- Serviço de internet: Clientes com 'Fiber optic' tendem a sair mais, possivelmente devido a expectativas não atendidas ou concorrência.
- Cobrança: Valores mais altos de cobrança mensal e total aumentam a chance de churn.
- Fatura digital: Clientes que utilizam paperless billing também apresentam maior propensão ao churn.
- Serviços adicionais: A ausência de suporte técnico e segurança online pode contribuir para o aumento do churn.

Recomendações para retenção:

1. Oferecer benefícios ou descontos para clientes com contrato mensal, incentivando a migração para contratos anuais ou bienais.
2. Melhorar a experiência dos clientes que utilizam 'Electronic check', promovendo métodos de pagamento automáticos e mais práticos.
3. Investir em qualidade e atendimento para usuários de 'Fiber optic', entendendo suas principais reclamações.
4. Oferecer pacotes personalizados ou descontos para clientes com cobranças mensais elevadas.
5. Promover serviços de suporte técnico e segurança online como diferenciais para aumentar o valor percebido.
6. Realizar campanhas de engajamento para clientes que utilizam fatura digital, mantendo contato próximo e oferecendo vantagens exclusivas.

Essas ações podem ajudar a reduzir o churn e aumentar a satisfação dos clientes mais propensos a sair.
"""

print(relatorio)